In [58]:
import numpy as np
import matlab.engine

mle = matlab.engine.start_matlab('-desktop')

In [59]:

tunableParams = {
    'current_height': 0.0,
    'current_vel':0.0,
    'current_accel':0.0,
    'current_pitch':0.0,
    'extension_length':0.0
}
results = mle.sim_the_model('Stoptime',40,'TunableParameters',tunableParams)

mle.quit()

In [71]:
print(results['velocity']['Data'])


[[0.0],[19.97460163380854],[39.920484004448646],[60.25136818515412],[80.97089729744556],[102.09905142782807],[123.55039761091325],[145.31004255183026],[167.387089654409],[189.78819211244365],[212.5171226510368],[235.57437931814206],[258.8917367329808],[282.3753668784644],[306.00515197834517],[329.75939025996007],[353.6063517118909],[377.37401608478496],[400.97880371375965],[424.3721774899823],[447.50142635477596],[470.1670866472309],[492.20342228502045],[513.542186724421],[534.1175513789145],[553.865631384159],[572.7258544849227],[589.8097566811296],[603.4505784819871],[613.4531604766767],[619.8497312336331],[623.3343053186117],[624.8619396008608],[623.7040498381637],[619.3813448692953],[612.130094792222],[602.7176101527579],[592.2743517611087],[582.2969481986066],[572.8146638212195],[563.786486452826],[555.1731845236193],[546.9400840276359],[539.0565221071492],[531.495264828027],[524.2320213005022],[517.2450402716393],[510.51477316638926],[504.0235908906089],[497.7555444413641],[491.6

In [72]:
def getApogee(result_matlab):
    velocity_data = result_matlab['velocity']['Data']
    velocity_time = result_matlab['velocity']['Time']
    height_data = result_matlab['height']['Data']
    height_time = result_matlab['height']['Time']

    #find time when apogee hit
    apogee_index = -1
    for i in range(0,len(velocity_data)):
        if (velocity_data[i][0]<=0 and i>1):
            apogee_index = i
            break
            
    if (apogee_index != -1):
        #find time when apogee hit
        apogee_time = velocity_time[apogee_index][0]
        return height_data[apogee_index][0]
    else:
        return -1 #INVALID APOGEE
    

In [73]:
print(getApogee(results))

7092.876819589577


In [74]:
print((tunableParams))

<class 'dict'>
